In [250]:
# Chase Abram
using Pkg
# Pkg.add("DataFrames")
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random
using Statistics

# Load data
df = DataFrame()
df = CSV.read("psetOne.csv", DataFrame)

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,1,1,1.55471,3.0,1,0,0,0.237685,0.0631155
2,1,1,1.55981,5.0,1,0,0,0.00203326,0.186073
3,1,1,1.85719,3.0,1,1,0,0.16455,0.0333083
4,1,1,1.99544,5.0,1,0,1,0.485538,0.117025
5,1,1,4.01799,7.5,1,0,1,0.582503,0.368032
6,1,1,4.14367,10.0,1,0,1,0.564396,0.442613
7,1,1,2.82341,10.0,0,0,1,0.275784,0.26417
8,1,1,6.55832,12.0,0,0,1,1.18369,0.528303
9,2,1,2.33037,5.0,1,0,0,0.316675,0.217231


In [2]:
# names(df)

# Get market t = 17
df17 = df[df."Market" .== 17, :]

# Store variables for market 17
X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)
p_given = df17."Price"
df17

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64,Float64
1,17,1,0.920128,3.0,1,0,0,0.0276024,0.0282533
2,17,1,1.97453,5.0,1,0,0,0.219283,0.296879
3,17,1,1.94548,3.0,1,1,0,0.150052,0.0497719
4,17,1,1.86088,5.0,1,1,0,0.387576,0.301157
5,17,1,1.98307,6.5,0,0,1,0.0714796,0.033719
6,17,1,3.8985,10.0,1,0,1,0.0523355,0.601297
7,17,1,2.86479,10.0,0,0,1,0.152532,0.0601887


In [289]:
# Shares
function s_logit(p, alpha, beta, X, xi)
    
    # Compute numerators for each s_j
    nums = exp.(X*beta - alpha.*p + xi)
    
    return nums./(1 + sum(nums))
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Shares
    s = s_logit(p, alpha, beta, X, xi)

    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                if j == k
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
    Om = zeros(size(br,1),size(br,1))
    
    for i in 1:size(Om,1)
        for j in 1:size(Om,2)
            if br[i,1] && br[j,1]
                # Brand2
                Om[i,j] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i,j] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i,j] = 1
            end
        end
    end
    
    return Om
end

# Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* -s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return Omega(p, alpha, beta, X, xi, br) \ s_logit(p, alpha, beta, X, xi) - p
end

# Jacobian of Omega^{-1}
function jac_inv_om(p, alpha, beta, X, xi, br)
    
    # Omega
    om = Omega(p, alpha, beta, X, xi, br)
    
    # Ownership
    oms = Omega_star(br)
    
    # Hessian
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    jac = zeros(length(p), length(p), length(p))
    
    # Fill in (this relies on rules of tensor calculus)
    # See https://math.stackexchange.com/questions/1471825/derivative-of-the-inverse-of-a-matrix
    for k in 1:size(jac,3)
        jac[:,:,k] = - inv(om) * (oms .* -sh[:,:,k]) * inv(om)
    end
    
    return jac
end

# Jacobian of fixed point expression
function J_fp_logit(p, alpha, beta, X, xi, br)
    
    # Shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian of shares
    sj = s_logit_jac(p, alpha, beta, X, xi)
    
    # Omega
    om = Omega(p, alpha, beta, X, xi, br)
    
    # Jacobian of 
    jacomi = jac_inv_om(p, alpha, beta, X, xi, br)
    
    # Initialize (Jacobian of Omega^{-1}*s)
    jac_inv_om_s = zeros(length(p), length(p))
    
     # Fill in
    for i in 1:length(p)
        for j in 1:length(p)
#             println("size in J_fp: ", size(jacomi[i,:,j]'*s))
            jac_inv_om_s[:,j] = jacomi[:,:,j]*s
        end
    end
    
    return jac_inv_om_s + om \ sj - I
end

# Solve for fixed point
function solve_fp_logit(p0, alpha, beta, X, xi, br, tol = 1e-14, maxiter = 1000)
    
    # Initialize price and update storage
    p = zeros(length(p0),1)
    pnew = zeros(length(p0),1)
    
    p .= p0
    
    # Initialize Jacobian
    J = zeros(length(p0), length(p0))
    
    # Iterator
    it = 0
    
    # Difference in updating
    diff = Inf
    
    # Continue until converged or maximum iterations
    while diff > tol && it < maxiter
        
        # Jacobian of FP expression
        J = J_fp_logit(p, alpha, beta, X, xi, br)
#         J = ForwardDiff.jacobian(x->fp_logit(x, alpha, beta, X, xi, br), p)
        
        # Get new
        pnew .= p .- J \ fp_logit(p, alpha, beta, X, xi, br)
        it += 1
        
        # Get difference
        diff = maximum(abs.(pnew .- p))
        
        # Update
        p .= pnew
    end
    
    println("Took ", it, " iterations")
    println("final fp_logit: ", fp_logit(p, alpha, beta, X, xi, br))
    return p
end

solve_fp_logit (generic function with 3 methods)

In [290]:
p_init = ones(size(X,1),1) .* 1.0
# p .= p_given
# p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'


# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1)) .* 0.0
# println("xi: ", xi)

# println("p: ", p)
# println("delta: ", X*beta - alpha .* p + xi)

s_logit(p_init, alpha, beta, X, xi)
println("shares: ", s_logit(p_init, alpha, beta, X, xi))
# println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p_init, alpha, beta, X, xi)
# println("shares_jac: ", s_logit_jac(p, alpha, beta, X, xi))

fp_logit(p_init, alpha, beta, X, xi, Br)
# s_logit_hess(p, alpha, beta, X, xi)

jac_inv_om(p_init, alpha, beta, X, xi, Br)

# Omega_star(Br)

# Omega(p, alpha, beta, X, xi, Br)

# fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)
# println("fp_logit: ", fp_logit(p, alpha, beta, X, xi, Br))
# fp_logit(p, alpha, beta, X, xi, Br)

J_fp_logit(p_init, alpha, beta, X, xi, Br)

out = solve_fp_logit(p_init, alpha, beta, X, xi, Br)
# println("out: ", out)
out
# println("eval fp_logit: ", fp_logit(out, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p_init, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))

shares: [0.00029341911289887894; 0.0021680902857082822; 0.00010794285928225993; 0.0007975958427155957; 0.0035745765708455634; 0.8746700481466118; 0.11837371870450338]
Took 10 iterations
final fp_logit: [0.0; 0.0; 0.0; 0.0; 4.440892098500626e-16; 2.220446049250313e-16; 2.220446049250313e-16]


7×1 Array{Float64,2}:
 0.3844741329790391
 0.38447413297903904
 0.35231945804382314
 0.3523194580438232
 1.7822602606218434
 1.7822602606218436
 1.7822602606218436

In [50]:
# NM: opam: [0.3720742469890362; 0.3724406061806239; 0.3447723065789802; 
# 0.3424326539927414; 2.4893064284464703; 2.5105275013368717; 4.366510826679992]

# BFGS: opam: [0.37346117252339095; 0.37346116923202516; 5.8690207005673365; 
# 0.34160603728869476; 2.5287477985474984; 2.5287470025674392; 2.528747016448347]

# From Sam: 
# 0.3938563345938473
# 0.3938563345938473
# 0.3496634377039666
# 0.3496634413963176
# 1.6632109730370337
# 1.6632109730370337
# 1.6632541435947288





In [52]:
# function f1(x, y, z) 
#     return (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
# end

# function g1(storage, x, y, z)
#     storage[1] = 2*(x[1] - 5 - y)
#     storage[2] = 2*(x[2] - 3 - z)
# end

# function g2(x,y,z)
#     out = zeros(2)
#     out[1] = 2*(x[1] - 5 - y)
#     out[2] = 2*(x[2] - 3 - z)
    
#     return out
# end

# function tester()
#     a = 1000
#     b = -1000
#     f(x) = f1(x,a,b)
# #     g!(storage, x) = g1(storage, x, a, b)
#     g!(x) = g2(x,a,b)
    
#     out = basic_newton_zero(f, g!, [-200.0, 1000.0])
    
# #     opt = optimize(f, g!, [100.0, -20.0], LBFGS())
# #     return opt, opt.minimizer, opt.minimum
#     return out
# end

# tester()



In [291]:
p_init = ones(size(X,1),1) .* 1.0
# p_init[1] = 50
# p_init[2] = 30
# p_init[3] = 90
# p_init[4] = -12
# p_init[5] = -40
# p_init[6] = 0
# p_init[7] = 1
s(x) = s_logit(x, alpha, beta, X, xi)
# println(s_logit_jac(p, alpha, beta, X, xi))
ForwardDiff.jacobian(s, p_init)

# ForwardDiff.gradient(fp, p)


# fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
# println(fp_logit_2_jac(p, alpha, beta, X, xi, Br))
# ForwardDiff.gradient(fp, p)

fp(x) = fp_logit(x, alpha, beta, X, xi, Br)
J_ex = J_fp_logit(p_init, alpha, beta, X, xi, Br)
println(J_ex)
# println(maximum(abs.(ForwardDiff.jacobian(fp, p) .- J_ex)))
ForwardDiff.jacobian(fp, p_init)
# println(inv(J_ex))

[-1.0002941431490249 -0.0021734402292599153 2.670152659561587e-7 1.9729907794210043e-6 8.842331212008132e-6 0.0021636471099870658 0.0002928177944541699; -0.0002941431490247394 -1.0021734402292601 2.6701526595611803e-7 1.9729907794207874e-6 8.842331212007264e-6 0.0021636471099865107 0.00029281779445412826; 2.6618422459923985e-7 1.966850168213987e-6 -1.0001080406943128 -0.000798318751241375 3.242787708615004e-6 0.0007934839903438462 0.00010738638057687655; 2.6618422459913143e-7 1.9668501682131197e-6 -0.00010804069431235051 -1.0007983187512415 3.242787708613269e-6 0.0007934839903434021 0.00010738638057682104; 25.571621885866207 188.95014865530817 9.407273969219885 69.51087509657573 -2.0570489605774886 -258.65135266152663 -35.00465407197606; 25.571621885866207 188.95014865530817 9.407273969219887 69.51087509657575 -1.0570489605773157 -259.65135266152714 -35.00465407197624; 25.571621885866207 188.95014865530823 9.407273969219887 69.51087509657573 -1.0570489605773143 -258.65135266152674 -36.

7×7 Array{Float64,2}:
 -1.00029       -0.00217344   2.67015e-7   …     0.00216365     0.000292818
 -0.000294143   -1.00217      2.67015e-7         0.00216365     0.000292818
  2.66184e-7     1.96685e-6  -1.00011            0.000793484    0.000107386
  2.66184e-7     1.96685e-6  -0.000108041        0.000793484    0.000107386
 25.5716       188.95         9.40727         -258.651        -35.0047
 25.5716       188.95         9.40727      …  -259.651        -35.0047
 25.5716       188.95         9.40727         -258.651        -36.0047

In [320]:
# Compute probabilities of agent i choosing product j
function pr(delta::AbstractVector{T}, X, sigma, zeta) where T
    
    # Initialize utilities
#     ubar = zeros(T, size(zeta,2), size(delta,1))
    
#     # Compute utilities
#     for i in 1:size(ubar, 1)
#         for j in 1:size(ubar, 2)
#             ubar[i,j] = delta[j] + X[j,:]' * sigma * zeta[:,i]
#         end
#     end
    
    # Initialize probabilities
#     p = zeros(T, size(zeta,2), size(delta,1))
    
    # Compute probabilities
#     for i in 1:size(p,1)
#         for j in 1:size(p,2)
# #             p[i,j] = exp(ubar[i,j])/(1 + sum(exp.(ubar[i,:])))
#             p[i,j] = exp(delta[j] + X[j,:]' * sigma * zeta[:,i]) ./(1 + sum(exp.(delta[:] + X[:,:] * sigma * zeta[:,i])))
#         end
#     end
    
#     return p
#     println("num: ", exp.(delta .+ X * sigma * zeta))
#     println("part of denom: ", sum(exp.(delta .+ X * sigma * zeta), dims = 2))
#     println("denom: ", 1 .+ sum(exp.(delta .+ X * sigma * zeta), dims = 1))
    return exp.(delta .+ X * sigma * zeta .- maximum(delta)) ./(exp(-maximum(delta)) .+ sum(exp.(delta .+ X * sigma * zeta .- maximum(delta)), dims = 1))
end



# Shares
function sHat(delta::AbstractVector{T}, X, sigma, zeta) where T
    
    # Compute probs
#     p = pr(delta, X, sigma, zeta)
#     println("p in s: ", p)
    # Initialize shares
#     s = zeros(T, size(delta))
    
    # Compute share
#     for j in 1:length(s)
#         s[j] = 1/size(zeta,2) * sum(p[:,j])
#     end
    
#     return s
#     return sum.(pr(delta, X, sigma, zeta))
#     return exp.(delta .+ X * sigma * zeta) /(1 .+ sum(exp.(delta .+ X * sigma * zeta), dims = 1))
    return mean(pr(delta, X, sigma, zeta), dims=2)
end


# Test cases
nI = 20
nJ = 3
nN = 5

delta_1 = zeros(3)
X_1 = zeros(3,nN)
sigma_1 = 0.0 .* I(size(X,2))
zeta_1 = zeros(nN, nI)

delta_2 = zeros(3)
delta_2[1] = 40
delta_2[2] = 20
delta_2[3] = 20
# delta_2[1] = 5
# delta_2[2] = 3
# delta_2[3] = 6
X_2 = zeros(nJ,nN)
sigma_2 = 0.0 .* I(size(X,2)).* 0.1
zeta_2 = zeros(nN, nI)

sigma_3 = 0.1 .* I(nN)
delta_3 = zeros(nJ)
X_3a = zeros(nJ,nN)

rng = MersenneTwister(123)
zeta_3 = randn(rng, nN, nI)

rng = MersenneTwister(123)
X_3b = randn(rng, size(X_3a))
X_3c = X_3b .* 10
X_3d = abs.(X_3c)


pr1 = pr(delta_1, X_1, sigma_1, zeta_1)
println("pr1: ", pr1)

# pr2 = pr(delta_2, X_2, sigma_2, zeta_2)
# println("pr2: ", pr2)

# pr3a = pr(delta_3, X_3a, sigma_3, zeta_3)
# println("pr3a: ", pr3a)

# pr3b = pr(delta_3, X_3b, sigma_3, zeta_3)
# println("pr3b: ", pr3b)

# pr3c = pr(delta_3, X_3c, sigma_3, zeta_3)
# println("pr3c: ", pr3c)
# println("mean pr3c: ", mean(pr3c, dims=1))

# pr3d = pr(delta_3, X_3d, sigma_3, zeta_3)
# println("pr3d: ", pr3d)
# println("mean pr3d: ", mean(pr3d, dims=1))

s1 = sHat(delta_1, X_1, sigma_1, zeta_1)
println("s1: ", s1)

s2 = sHat(delta_2, X_2, sigma_2, zeta_2)
println("s2: ", s2)

s3a = sHat(delta_3, X_3a, sigma_3, zeta_3)
println("s3a: ", s3a)

s3b = sHat(delta_3, X_3b, sigma_3, zeta_3)
println("s3b: ", s3b)

s3c = sHat(delta_3, X_3c, sigma_3, zeta_3)
println("s3c: ", s3c)

s3d = sHat(delta_3, X_3d, sigma_3, zeta_3)
println("s3d: ", s3d)




pr1: [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25; 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
s1: [0.25; 0.25; 0.25]
s2: [0.9999999958776925; 2.0611536139418488e-9; 2.0611536139418488e-9]
s3a: [0.25; 0.25; 0.25]
s3b: [0.24799721478422257; 0.24544702592073464; 0.2510010702628024]
s3c: [0.26664175778642857; 0.2022947915585826; 0.2287346411240403]
s3d: [0.2402281349379146; 0.208663135975463; 0.22673593959852845]


In [325]:
# Jacobian of shares
function sHat_jac(delta, X, sigma, zeta)
    # Get probabilities
    p = pr(delta, X, sigma, zeta)
    
#     # Get shares
#     s = sHat(delta, X, sigma, zeta)
    
#     # Initialize Jacobian
    sj = zeros(size(delta,1), size(delta,1))
    
#     # Fill in
    for j in 1:size(sj,1)
        for k in 1:size(sj,2)
            if j == k
                sj[j,k] = 1/size(zeta,2) * sum(p[:,j] .* (1 .- p[:,j]))
            else
                sj[j,k] = 1/size(zeta,2) * sum(-p[:, j] .* p[:, k])
            end
        end
    end
    
#     println("pr: ", pr(delta, X, sigma, zeta))
#     println("outer product pr: ", 1/size(zeta,2).*pr(delta, X, sigma, zeta)*pr(delta, X, sigma, zeta)')
    return sj
#     return 1/size(zeta,2) .* (-pr(delta, X, sigma, zeta)*pr(delta, X, sigma, zeta)' + I(length(delta)) .* diag(sum(pr(delta, X, sigma, zeta),dims=2)))
#     return  mean(-pr(delta, X, sigma, zeta)*pr(delta, X, sigma, zeta)' + I(length(delta)) .* diag(sum(pr(delta, X, sigma, zeta),dims=2)), dims=)
end

# Jacobian of log of shares
function sHat_log_jac(delta, X, sigma, zeta)
    # shares
#     s = sHat(delta, X, sigma, zeta)
    
#     # jacobian of shares
#     sj = sHat_jac(delta, X, sigma, zeta)
    
#     # initialize jacobian of log of share
#     slj = zeros(size(delta,1), size(delta,1))
    
#     # Fill in
#     for j in 1:size(slj, 1)
#         for k in 1:size(slj,2)
#             slj[j,k] = sj[j,k]/s[k]
#         end
#     end
    
#     return slj
    return sHat_jac(delta, X, sigma, zeta) ./ sHat(delta, X, sigma, zeta)
end

s1_jac = sHat_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_jac: ", s1_jac)
s1_log_jac = sHat_log_jac(delta_1, X_1, sigma_1, zeta_1)
println("s1_log_jac: ", s1_log_jac)

s1_jac: [4.505605635274746e-5 -2.851023957051512e-5 -2.0682069878535107e-5; -2.851023957051512e-5 4.073479875993136e-5 -2.81555683653301e-5; -2.0682069878535107e-5 -2.81555683653301e-5 5.6342132315651366e-5]
s1_log_jac: [0.00014838390832173115 -9.389327688915542e-5 -6.811262700698069e-5; -0.00010817268057036979 0.00015455472983514943 -0.00010682699791165007; -5.687949436414532e-5 -7.74329891331231e-5 0.00015495122182320993]


In [322]:
function sHat_inv(s, X, sigma, zeta, tol = 1e-14, maxiter = 1e6)
    
    # Initialize
    delta = zeros(size(X,1))
#     delta_new = zeros(length(delta))
    shat = zeros(length(delta))
    inc = zeros(length(delta))
    
    log_s = log.(s)
    
    diff = maximum(log_s - log.(sHat(delta, X, sigma, zeta)))
    it = 0
#     println("delta init: ", delta)
    while diff > tol && it < maxiter
#         println("it: ", it)
        
#         println("delta 1: ", size(delta))
        shat = sHat(delta, X, sigma, zeta)
#         println("shat: ", shat)
        
        if diff > 0
#             println("Contraction")
#             println("log diff: ", (log.(s) - log.(shat))[:,1])
            inc .= (log_s .- log.(shat))[:,1]
        else
#             println("Newton")
            
#             println("slj: ", slj)
            inc = (-sHat_log_jac(delta, X, sigma, zeta) \ (log.(shat) - log_s))[:,1]
        end
        
        delta += inc
#         println("delta 2: ", size(delta))
#         println("delta post: ", delta)
        diff = maximum(abs.(inc))
#         diff =
#         println("diff: ", diff)
        it += 1
    end
    println("Loop exited")
    
    println("final it: ", it)
    println("final diff: ", diff)
    println("final delta: ", delta)
    
    return delta
end

println("s1_inv: ", sHat_inv(s1, X_1, sigma_1, zeta_1))
println("s2_inv: ", sHat_inv(s2, X_2, sigma_2, zeta_2))
println("s3a_inv: ", sHat_inv(s3a, X_3a, sigma_3, zeta_3))
# println("s3b_inv: ", sHat_inv(s3b, X_3b, sigma_3, zeta_3))
# println("s3c_inv: ", sHat_inv(s3c, X_3c, sigma_3, zeta_3))
# println("s3d_inv: ", sHat_inv(s3d, X_3d, sigma_3, zeta_3))

Loop exited
final it: 0
final diff: 0.0
final delta: [0.0, 0.0, 0.0]
s1_inv: [0.0, 0.0, 0.0]
Loop exited
final it: 1000000
final diff: 9.999975002179464e-7
final delta: [13.815513553773915, -6.184486446226085, -6.184486446226085]
s2_inv: [13.815513553773915, -6.184486446226085, -6.184486446226085]
Loop exited
final it: 0
final diff: 0.0
final delta: [0.0, 0.0, 0.0]
s3a_inv: [0.0, 0.0, 0.0]


In [324]:
delta_1 = ones(3)
# delta_2 = 2
# delta_3 = 40
X_1 = 10 .* randn(3,nN)
sigma_1 = 0.1 .* I(size(X,2))
zeta_1 = randn(nN, 10000)




shl(x) = log.(sHat(x, X_1, sigma_1, zeta_1))
println(sHat_log_jac(delta_1, X_1, sigma_1, zeta_1))
ForwardDiff.jacobian(shl, delta_1)

# sh(x) = sHat(x, X_1, sigma_1, zeta_1)
# println(sHat_jac(delta_1, X_1, sigma_1, zeta_1))
# ForwardDiff.jacobian(sh, delta_1)

[0.36005928044241853 -0.20727861021079905 -0.10810929994010603; -0.23880179320903708 0.5816076677897304 -0.13293080157358172; -0.09027991705597758 -0.09635419430411575 0.09873985569211599]


3×3 Array{Float64,2}:
  0.360059   -0.207279   -0.108109
 -0.238802    0.429526   -0.132931
 -0.0902799  -0.0963542   0.26366